In [1]:
import numpy as np 
import scipy.io 
mat = scipy.io.loadmat('BearBullproblem.mat')
bull = mat['pbull']
bear = mat['pbear']
prices = mat['p'][0]

In [8]:
T = len(prices)

In [41]:
def p_transition(x, y):
    """ Returns the probability of transiting from state x to y. """
    if x == "bear" and y =="bear":
        return 0.8
    elif x == "bear" and y == "bull":
        return 0.2
    elif x == "bull" and y =="bull":
        return 0.7
    elif x == "bull" and y == "bear":
        return 0.3

In [42]:
def p_emission(x, y, z):
    """ Returns the probability p(v_t = x | h_t = y, v_{t-1} = z) """
    return bear[x][z] if y == "bear" else bull[x][z]

In [43]:
alphas = {"bear":[0.5],"bull":[0.5]}

In [79]:
for t in range(1, T):
    for a in ["bear", "bull"]:
        alpha = p_emission(prices[t], a, prices[t-1])
        tmp = 0
        for b in ["bear", "bull"]:
            tmp += p_transition(b, a) * alphas[b][t - 1]
        alphas[a] += [alpha * tmp]
    n = alphas["bear"][t] + alphas["bull"][t]
    alphas["bear"][t] /= n
    alphas["bull"][t] /= n
        

In [142]:
# for i in range(0, T):
#     a, b = alphas["bear"][i], alphas["bull"][i]
#     print("{} - {} - {}".format(a, b, a > b))

In [83]:
# Probability of being in state bull and bear at the last time step
p_bull_t = alphas["bull"][-1] / (alphas["bull"][-1] + alphas["bear"][-1])
p_bear_t = 1 - p_bull

In [87]:
# Predicted probability of being in state bear and bull after the observed time steps
p_bear = p_bear_t * p_transition("bear", "bear") + p_bull_t * p_transition("bull", "bear")
p_bull = 1 - p_bear

In [124]:
price = prices[-1]

In [143]:
p_values = []
for next_price in range(0, 100):
    prob = p_bear * p_emission(next_price, "bear", price) + p_bull * p_emission(next_price, "bull", price)
    p_values.append(prob)

In [150]:
p_values[89]

4.211641658081237e-05

In [168]:
gain = 0
gain_sq = 0
for next_price in range(0, 100):
    gain += p_values[next_price] * (next_price - price)
    gain_sq += p_values[next_price] * ((next_price - price) ** 2)
std = np.sqrt(gain_sq - gain ** 2)

In [169]:
gain

1.4323286305645822

In [170]:
std

8.54097765342951